In [13]:
import numpy as np
import pandas as pd

In [14]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False


def convertToNumber(strVal):
    lowerStrVal = strVal.lower()
    print("original value", lowerStrVal)
    if lowerStrVal.endswith("k"):
        return int(float(strVal[0:len(strVal)-1]) * 1000)
    elif lowerStrVal.endswith("/month"):
        return int(float(strVal[0:len(strVal)-6]) * 12)
    elif lowerStrVal.endswith("/year"):
        return int(strVal[0:len(strVal)-5])
    elif isfloat(lowerStrVal):
        return int(float(lowerStrVal))
    else:
        return 0

def standardizeString(val):
    strVal = str(val)
    return strVal.title()

In [15]:
print(standardizeString(3.926))
print(standardizeString("925.3y"))
print(standardizeString("sdfdfs dsSFs"))
print(standardizeString("ABC dsfs dfd"))
print(standardizeString("bcA A"))

3.926
925.3Y
Sdfdfs Dssfs
Abc Dsfs Dfd
Bca A


In [19]:
salaryInfo = pd.read_csv(
    './salary_infomation.csv', encoding="ISO-8859-1").head(6)

print(salaryInfo.to_string())

for index, row in salaryInfo.iterrows():
    salaryInfo.at[index, 'Company'] = standardizeString(row['Company'])
    # row['Campany'] = standardizeString(row['Campany'])
    salaryInfo.at[index, 'Job'] = standardizeString(row['Job'])
    salaryInfo.at[index, 'Area'] = standardizeString(row['Area'])
    # print(row['education'])
    # print(row)
    salaryInfo.at[index, 'Education'] = standardizeString(row['Education'])

print(salaryInfo.to_string())


    Company        Offer Time                           Job                Area    Base  Equity Bonus Sign-on Full Time/Intern Title/Level Education  New Grad Working Experience Return
0    Oracle  2021 4th quarter             Software Engineer  Great Seattle Area    160k    300k     0     40k        full time    IC & IC3        MS     False              2.5 y    NaN
1     Chewy  2021 4th quarter             Software Engineer  Great Seattle Area    154k    181k   15%     15k             full  IC& SDE II        MS     False                 3y    NaN
2     Adobe  2021 4th quarter                  Data Science            Bay Area    160k  187500   15%     16k             full   IC &DS II        MS     False               1.3y    NaN
3      Wish  2021 4th quarter             Software Engineer  Great Seattle Area    236K    750k     0     70k             full     IC & L4        MS     False                 4y    NaN
4  FaceBook  2021 4th quarter  Machine Learning Engineering  Great Seattle 

In [6]:
WorldRecords = pd.read_csv(
    './WorldRecords_1.csv', encoding="ISO-8859-1")
# print(WorldRecords.to_string())

In [7]:
dfTimeEvent = WorldRecords[(WorldRecords["Type"] == "time")]
# get top 5 items of this dateframe
dfTimeEvent = dfTimeEvent.head(5)
# print("dfTimeEvent", dfTimeEvent)

for index, row in dfTimeEvent.iterrows():
    record = row['Record']
    res = convertToNumber(str(record))
    print('converted value', res)
    print()


original value 100.6k
converted value 100600

original value 1003.35/month
converted value 12040

original value 102202/year
converted value 102202

original value 9.95
converted value 9

original value 993
converted value 993



ParserError: Error tokenizing data. C error: Expected 1 fields in line 128, saw 2
